In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
hkayan_industrial_robotic_arm_imu_data_casper_1_and_2_path = kagglehub.dataset_download('hkayan/industrial-robotic-arm-imu-data-casper-1-and-2')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
DATASET_PATH="/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/"

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_Acc_Gyro_20Hz.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_hitting_arm.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_20Hz_freq_drop_2.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_hitting_platform.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_10Hz.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/right_arm.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_2Hz.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/quaternions_no_idle.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_extra_weigth.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_20Hz_freq_drop.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/quaternions_20Hz.csv
/kaggle/input/industrial-robotic-arm-imu-data-casper-1-and-2/IMU_earthquake.csv
/kaggle/input/industrial-robo

In [ ]:




#nos decidimos detectar apenas anomalias dos braços, entao
# desconsideraremos erros de sensores (freq_drop)
import re

normalPattern = "IMU_(.*)Hz\.csv"

normal = ("IMU_10Hz.csv", ("label", 0))
faulty =("IMU_hitting_platform.csv", ("label", 1))

df = pd.read_csv(DATASET_PATH + normal[0])
df['label'] = 0

faultydf = pd.read_csv(DATASET_PATH+faulty[0])
faultydf['label'] = 1



In [ ]:
df.shape, faultydf.shape

((874937, 12), (14967, 12))

In [ ]:
df.isnull().values.any()
# Sem NAs

False

# PREPARAÇÂO DOS DADOS
- convertendo o timestamp de ns pra s

In [ ]:
#ns pra s
df['time'] = df['time'].map(lambda x: x/1e9)
faultydf['time'] = faultydf['time'].map(lambda x: x/1e9)

print(f"A frequência é {1/df['time'].diff().mean():.2f}Hz")

A frequência é 9.98Hz


In [ ]:
diffs = df['time'].diff()
type(diffs)

pandas.core.series.Series

In [ ]:
print(f"Duração da amostragem: {(df['time'].iloc[-1] - df['time'].iloc[0])/3600:.2f}h")
print(f"Duração da amostragem (Anomalia): {(faultydf['time'].iloc[-1] - faultydf['time'].iloc[0])/3600:.2f}h")

Duração da amostragem: 24.34
Duração da amostragem (Anomalia): 0.42


In [ ]:
#remocao de duplicatas
ilenNormal, ilenFaulty = df.shape[0], faultydf.shape[0]
df.drop_duplicates(), faultydf.drop_duplicates()
ilenNormal - df.shape[0], ilenFaulty - faultydf.shape[0]


(0, 0)